In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import warnings

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
def previous_app(nan_as_category = True):
    df_prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
    cat_cols = [col for col in df_prev.columns if df_prev[col].dtypes == 'O']
    num_cols = [col for col in df_prev.columns if df_prev[col].dtypes != 'O']
    
    # days 365243 values to nan
    df_prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    df_prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    df_prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    df_prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    df_prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    
    # XNA, XAP to nan for cat_cols.
    na = ['XNA', 'XAP']
    for col in cat_cols:
        for n in na:
            df_prev.loc[df_prev[col] == n, col] = np.nan
    
    # delete columns columns that do not contain information or missing values over 80 percent of the entire data
    del_cols = ['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_FIRST_DRAWING',
                'NAME_CASH_LOAN_PURPOSE', 'CODE_REJECT_REASON', 'FLAG_LAST_APPL_PER_CONTRACT', 
                'NFLAG_LAST_APPL_IN_DAY', 'SELLERPLACE_AREA']
    df_prev.drop(del_cols, axis=1, inplace=True)
    
    # Feature Engineering
    # X-sell approved & Walk-in Approved
    df_prev['NEW_X_SELL_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_PRODUCT_TYPE'] == 'x-sell') & 
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_X_SELL_APPROVED'] = 1
    df_prev['NEW_WALK_IN_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_PRODUCT_TYPE'] == 'walk-in') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_WALK_IN_APPROVED'] = 1
    
    # Customer status approved
    df_prev['NEW_REPEATER_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_CLIENT_TYPE'] == 'Repeater') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_REPEATER_APPROVED'] = 1
    df_prev['NEW_NEWCUST_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_CLIENT_TYPE'] == 'New') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_NEWCUST_APPROVED'] = 1
    df_prev['NEW_REFRESHED_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_CLIENT_TYPE'] == 'Refreshed') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_REFRESHED_APPROVED'] = 1
    
    # Purpose of application approved
    df_prev['NEW_CARDS_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_PORTFOLIO'] == 'Cards') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_CARDS_APPROVED'] = 1
    df_prev['NEW_CASH_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_PORTFOLIO'] == 'Cash') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_CASH_APPROVED'] = 1
    df_prev['NEW_POS_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_PORTFOLIO'] == 'POS') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_POS_APPROVED'] = 1
    
    # Interest approved
    df_prev['NEW_HIGH_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'high') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_HIGH_APPROVED'] = 1
    df_prev['NEW_MIDDLE_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'middle') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_MIDDLE_APPROVED'] = 1
    df_prev['NEW_LOWACTION_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'low_action') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_LOWACTION_APPROVED'] = 1
    df_prev['NEW_LOWNORMAL_APPROVED'] = 0
    df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'low_normal') &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_LOWNORMAL_APPROVED'] = 1
    
    # Application hour convert to categorical
    df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] >= 0) & 
                (df_prev['HOUR_APPR_PROCESS_START'] <= 6), 'NEW_APP_DAY_TIME'] = 'night'
    df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] > 6) & 
                (df_prev['HOUR_APPR_PROCESS_START'] <= 12), 'NEW_APP_DAY_TIME'] = 'morning'
    df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] > 12) & 
                (df_prev['HOUR_APPR_PROCESS_START'] <= 18), 'NEW_APP_DAY_TIME'] = 'afternoon'
    df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] > 18) & 
                (df_prev['HOUR_APPR_PROCESS_START'] < 24), 'NEW_APP_DAY_TIME'] = 'evening'
    df_prev.drop('HOUR_APPR_PROCESS_START', axis=1, inplace=True)
    
    # Client apply with someone
    df_prev.loc[df_prev['NAME_TYPE_SUITE'] == 'Unaccompanied', 'NEW_ACCOMPANIED'] = 0
    df_prev.loc[df_prev['NAME_TYPE_SUITE'] != 'Unaccompanied', 'NEW_ACCOMPANIED'] = 1
    df_prev.loc[df_prev['NAME_TYPE_SUITE'].isnull(), 'NEW_ACCOMPANIED'] = np.nan
    df_prev.drop('NAME_TYPE_SUITE', axis=1, inplace=True)
    
    # credit requested / credit given ratio 
    df_prev['NEW_APP_CREDIT_RATIO'] = df_prev['AMT_APPLICATION'].div(df_prev['AMT_CREDIT']).replace(np.inf, 0)
    # loan installment / credit amount ratio
    df_prev['NEW_ANNUITY_CREDIT_RATIO'] = df_prev['AMT_ANNUITY'] / df_prev['AMT_CREDIT']
    # credit amount / goods price ratio
    df_prev['NEW_CREDIT_GOODS_RATIO'] = df_prev['AMT_CREDIT'].div(df_prev['AMT_GOODS_PRICE']).replace(np.inf, 0)
    # interest amount
    df_prev['NEW_AMT_INTEREST'] = df_prev['CNT_PAYMENT'] * df_prev['AMT_ANNUITY'] - df_prev['AMT_CREDIT']
    # interest ratio
    df_prev['NEW_INTEREST_RATIO'] =  df_prev['NEW_AMT_INTEREST'] / df_prev['AMT_CREDIT']
    # needed amount / credit amount (belki silinir)
    df_prev['NEW_AMT_NEEDED_CREDIT_RATIO'] = (df_prev['AMT_GOODS_PRICE'] - df_prev['AMT_DOWN_PAYMENT']) / \
    df_prev['AMT_CREDIT']
    
    # risk assessment via NEW_CREDIT_GOODS_RATIO
    df_prev.loc[df_prev['NEW_CREDIT_GOODS_RATIO'] >= 0.80, 'NEW_CREDIT_GOODS_RISK'] = 1
    df_prev.loc[df_prev['NEW_CREDIT_GOODS_RATIO'] < 0.80, 'NEW_CREDIT_GOODS_RISK'] = 0
    
    # risk to approved
    df_prev['NEW_RISK_APPROVED'] = 0
    df_prev.loc[(df_prev['NEW_CREDIT_GOODS_RISK'] == 1) &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_RISK_APPROVED'] = 1
    
    # non risk to approved
    df_prev['NEW_NONRISK_APPROVED'] = 0
    df_prev.loc[(df_prev['NEW_CREDIT_GOODS_RISK'] == 0) &
                (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_NONRISK_APPROVED'] = 1
    
    # Application weekdays cycle encoding
    df_prev['WEEKDAY_APPR_PROCESS_START'] = df_prev['WEEKDAY_APPR_PROCESS_START'].map({
        'MONDAY': 1,'TUESDAY': 2,'WEDNESDAY': 3,'THURSDAY': 4,'FRIDAY': 5,'SATURDAY': 6,'SUNDAY': 7})
    df_prev['NEW_WEEKDAY_SIN'] = np.sin(2 * np.pi * df_prev['WEEKDAY_APPR_PROCESS_START']/7)
    df_prev['NEW_WEEKDAY_COS'] = np.cos(2 * np.pi * df_prev['WEEKDAY_APPR_PROCESS_START']/7)
    df_prev.drop('WEEKDAY_APPR_PROCESS_START', axis=1, inplace=True)
            
    # Rare encoding
    a = ['Auto Accessories','Jewelry','Homewares', 'Medical Supplies', 'Vehicles', 'Sport and Leisure',
         'Gardening', 'Other','Office Appliances', 'Tourism', 'Medicine', 'Direct Sales', 'Fitness',
         'Additional Service','Education', 'Weapon', 'Insurance', 'House Construction','Animals']
    df_prev["NAME_GOODS_CATEGORY"] = df_prev["NAME_GOODS_CATEGORY"].replace(a, 'others')

    b = ['Channel of corporate sales', 'Car dealer']
    df_prev["CHANNEL_TYPE"] = df_prev["CHANNEL_TYPE"].replace(b, 'Other_Channel')

    c = ['Auto technology', 'Jewelry', 'MLM partners', 'Tourism']
    df_prev["NAME_SELLER_INDUSTRY"] = df_prev["NAME_SELLER_INDUSTRY"].replace(c, 'Others')
    
    d = ['Non-cash from your account','Cashless from the account of the employer']
    df_prev["NAME_PAYMENT_TYPE"] = df_prev["NAME_SELLER_INDUSTRY"].replace(d, 'Others')
    
    # One hot encoder
    new_df_prev, new_cat_cols = one_hot_encoder(df_prev, nan_as_category = True)
    
    # Getting to all the cat cols
    origin_bin_cols = [col for col in df_prev.columns if (df_prev[col].dtypes != 'O') & (df_prev[col].nunique() == 2)]
    all_cat_cols = new_cat_cols + origin_bin_cols
    
    # Getting to the num cols
    # x_cols = ['SK_ID_PREV','SK_ID_CURR', 'DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION']
    # new_num_cols = [col for col in new_df.columns if (col not in all_binary_cols) and (col not in x_cols)]
    # num_aggregations = {}
    # for num in new_num_cols:
       # num_aggregations[num] = ['min', 'max', 'mean', 'median']
    
    
    # Previous app num features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean', 'median'],
        'AMT_APPLICATION': ['min', 'max', 'mean', 'median'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'median'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'median'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'median'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean', 'median'],
        'DAYS_DECISION': ['min', 'max', 'mean', 'median'],
        'CNT_PAYMENT': ['min', 'max', 'mean', 'median'],
        'NEW_APP_CREDIT_RATIO': ['min', 'max', 'mean', 'median'],
        'NEW_ANNUITY_CREDIT_RATIO': ['min', 'max', 'mean', 'median'],
        'NEW_CREDIT_GOODS_RATIO': ['min', 'max', 'mean', 'median'],
        'NEW_AMT_INTEREST': ['min', 'max', 'mean', 'median'],
        'NEW_INTEREST_RATIO': ['min', 'max', 'mean', 'median'],
        'NEW_AMT_NEEDED_CREDIT_RATIO': ['min', 'max', 'mean', 'median'],
        'NEW_WEEKDAY_SIN': ['min', 'max', 'mean', 'median'],
        'NEW_WEEKDAY_COS': ['min', 'max', 'mean', 'median']
    }
    
    # Previous app cat features
    cat_aggregations = {}
    for cat in all_cat_cols:
        cat_aggregations[cat] = ['mean']
        
    final_prev_df = new_df_prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    final_prev_df.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in final_prev_df.columns.tolist()])
    
    # Approved App - only num features
    approved = new_df_prev[new_df_prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['PREV_APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    final_prev_df = final_prev_df.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # refused App - only numerical features
    refused = new_df_prev[new_df_prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['PREV_REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    final_prev_df = final_prev_df.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, new_df_prev
    gc.collect()
    return final_prev_df

In [ ]:
prev_df = previous_app(nan_as_category = True)
prev_df.head()

In [ ]:
prev_df.shape